In [1]:
'''
Coding: PYTHON UTF-8
Created On: 2023-04-28 12:39:30
Author: Putu Hendra Widyadharma
=== filter .pha using wadati result from makeWADATI 
'''

'\nCoding: PYTHON UTF-8\nCreated On: 2023-04-28 12:39:30\nAuthor: Putu Hendra Widyadharma\n=== filter .pha using wadati result from makeWADATI \n'

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from localfunction import *
pd.options.mode.chained_assignment = None

In [2]:
path = 'G:\\My Drive\\Tomography\\090523\\arr-bmkgcat-run-sum2-08092023\\'
fname = 'arrivals_sum_8P.dat'
fname2 = 'temppha_arrivals_sum_8P.txt' #temppha
fname3 = 'tempwadati_arrivals_sum_8P.txt' #tempwadati
deviation = 9.5

In [5]:
df = readabsolute(path+fname)
dfphase = pd.read_csv(path+fname2,delim_whitespace=True)
dfwadati = pd.read_csv(path+fname3,delim_whitespace=True)

In [6]:
# hitung dan buat persamaan pembatas 
slope, intercept = np.polyfit(dfwadati['tp'].astype(float),dfwadati['ts-tp'].astype(float),1)

# predict function
predict = np.poly1d([slope, intercept])
# predict with deviation
predictup = np.poly1d([slope,intercept+deviation])
predictdown = np.poly1d([slope,intercept-deviation])

# hitung R^2 menguji kecocokan regresi
r2 = r2_score(dfwadati['ts-tp'],predict(dfwadati['tp']))
print('VP/VS = ' + str(slope + 1))
print(f"R-square = {r2}")

VP/VS = 1.7351618381049088
R-square = 0.9187907202684593


In [7]:
### dfwadati processing
# fungsi seleksi data berdasarkan batas atas dan bawah deviasi
def selection_data(row):
    if (row['ts-tp'] <= predictup(row['tp'])) and (row['ts-tp'] >= predictdown(row['tp'])):
        return True
    else:
        return False
# apply perhitungan
dfwadati['result'] = dfwadati.apply(selection_data,axis=1)
# select wadati hanya pada event yang perlu diubah
dfwadati = dfwadati[dfwadati['event_id'].isin(dfwadati[dfwadati['result'] == False]['event_id'].unique())]

In [8]:
# index search untuk event_id yang perlu disesuaikan menurut wadati 
editable_idx = df[df[0] == '#'][14].astype('int64').isin(dfwadati['event_id'].unique())
editable_idx = editable_idx[editable_idx==True].index
editable_idx

Int64Index([    51,    858,    869,    908,    974,   1068,   1235,   1410,
              1813,   1937,
            ...
            238894, 239446, 241330, 258350, 258738, 259914, 262620, 263901,
            264001, 265772],
           dtype='int64', length=272)

In [9]:
# buat tempat data hasil
df_result = pd.DataFrame([],columns = df.columns)

# pha index iterable
idx = df[df[0] == '#'].index

for idx_ in range (len(idx)):
# for idx_ in [1099]:
# for idx_ in [0,1,2,3]:
    # buat data per kejadian gempa
    if idx_ == len(idx)-1:
        tempdf = df.iloc[idx[idx_]::]
    else:
        tempdf = df.iloc[idx[idx_]:idx[idx_+1],:]
    # cek index header terhadap editable_idx
    if tempdf.index[0] in editable_idx:
        # proses delete data fasa S stasiun yang ts-tp diluar batas deviasi
        for i, row_ in tempdf.iterrows():
            # cari fasa S dan list stasiun yang masuk dalam membuat hasil wadati buruk 
            if row_[3]=='S' and row_[0] in (dfwadati[(dfwadati['event_id'] == int(tempdf.iloc[0,14])) & (dfwadati['result'] == False)]['sta'].to_list()):
                print(f'event: {tempdf.iloc[0,14]} -- index:{row_.name} --> {row_[0]} Phase:{row_[3]} dropped')
                tempdf.drop(row_.name,inplace=True)
    else:
        pass
    # masukkan hasil ke df hasil
    df_result = pd.concat([df_result,tempdf])

event: 4 -- index:58 --> PDSI Phase:S dropped
event: 4 -- index:63 --> RGRI Phase:S dropped
event: 49 -- index:864 --> PPI Phase:S dropped
event: 50 -- index:874 --> TSI Phase:S dropped
event: 53 -- index:913 --> LHMI Phase:S dropped
event: 53 -- index:915 --> BSI Phase:S dropped
event: 58 -- index:995 --> LWLI Phase:S dropped
event: 63 -- index:1077 --> TNGI Phase:S dropped
event: 74 -- index:1254 --> PPBI Phase:S dropped
event: 83 -- index:1419 --> TSI Phase:S dropped
event: 104 -- index:1824 --> BKNI Phase:S dropped
event: 104 -- index:1826 --> PPI Phase:S dropped
event: 104 -- index:1829 --> RGRI Phase:S dropped
event: 111 -- index:1946 --> TNGI Phase:S dropped
event: 118 -- index:2062 --> KSI Phase:S dropped
event: 118 -- index:2064 --> PPI Phase:S dropped
event: 118 -- index:2066 --> MNAI Phase:S dropped
event: 120 -- index:2108 --> PPBI Phase:S dropped
event: 132 -- index:2311 --> MNAI Phase:S dropped
event: 133 -- index:2319 --> KCSI Phase:S dropped
event: 143 -- index:2474 -->

In [10]:
# perbaiki df_result dan simpan dalam bentuk pha
df_result.sort_index(inplace = True)
df_result.reset_index(inplace = True, drop = True)
df2dat(df_result,evnum = 1, path = path, fname=Path(fname).stem+'_wadatifilter.dat')
print("== data filter")
readeventphase(path+Path(fname).stem+'_wadatifilter.dat')

Output finish: arrivals_sum_8P_wadatifilter.dat at G:\My Drive\Tomography\090523\arr-bmkgcat-run-sum2-08092023\
== data filter
! arrivals_sum_8P_wadatifilter.dat
---------------------------
start time        : 2008-11-01 13:04:38.700000
end time          : 2023-02-01 22:15:12.500000
minimum magnitude : 2.5
maximum magnitude : 8.2
minimum depth     : 1 km
maximum depth     : 750 km
station used      : 155
---------------------------
Event :  10817
Phase : 257314
-- P  : 209449
-- S  :  47487
-- Sg :    117
-- Pg :    170
-- Pn :     67
-- pP :     23
-- sP :      1
---------------------------
